In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [10]:
# path = DATA_DIR/'cairns_gtfs.zip'
path = DATA_DIR/'other_feeds'/'gtfs.zip'
#path = DATA_DIR/'other_feeds'/'seq_20170310_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20170213


In [11]:
v = gt.time_it(feed.validate)()
v

Timing validate
2017-04-16 12:27:34.734783 Began process
2017-04-16 12:27:49.937048 Finished in 0.25 min


,message_type,message,table,rows
1,error,arrival_time malformed,stop_times,"[675, 676, 677, 678, 679, 680, 681, 682, 683, ..."
2,error,departure_time malformed,stop_times,"[675, 676, 677, 678, 679, 680, 681, 682, 683, ..."
3,error,block_id unduplicated,trips,"[18079, 18167, 18286, 18410, 18503, 18880, 189..."
0,warning,ots_trip_short_name is not a valid column name,trips,[]


In [14]:

f = feed.stop_times.copy()
f.ix[675:676]


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
674,6777785,11:06:00,11:06:00,11025,75,NaN,0,0,23.0183,1
675,6777786,9:45:00,9:45:00,94,1,NaN,0,0,NaN,1
676,6777786,9:46:29,9:46:29,10957,2,NaN,0,0,0.6110,0


In [ ]:
f[['arrival_time', 'departure_time']].iloc[5].isnull().any()

In [ ]:
f = feed.stop_times.copy()
f

In [ ]:
feed.agency.columns

In [ ]:
feed.describe()

In [ ]:
report = feed.validate()

In [ ]:
report

In [ ]:
f = feed.stop_times.copy()
i = 10
f.ix[9:11]

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()